In [76]:
# http://metacyc.org/META/class-instances?object=Compounds
# Smart Table
# Add the following columns
# export with weblinks

In [1]:
import sys,os
sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
from metatlas import metatlas_objects as metob
import csv


curr_ld_lib_path = ''

os.environ['LD_LIBRARY_PATH'] = curr_ld_lib_path + ':/project/projectdirs/openmsi/jupyterhub_libs/boost_1_55_0/lib' + ':/project/projectdirs/openmsi/jupyterhub_libs/lib'
import sys
# sys.path.remove('/anaconda/lib/python2.7/site-packages')
sys.path.append('/global/project/projectdirs/openmsi/jupyterhub_libs/anaconda/lib/python2.7/site-packages')
sys.path.insert(0,'/project/projectdirs/openmsi/projects/meta-iq/pactolus/pactolus' )

from rdkit import Chem
# from rdkit.Chem.rdMolDescriptors import ExactMolWt
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem


In [2]:
"""
Create a list of dictionaries.  Each item in the list will be a dict containin the metacyc information.
"""
datafile = 'Selected_rows_from_All_instances_of_Compounds_in_MetaCyc.txt'
with open(datafile, 'rUb') as f:
    metacyc_compounds = list(csv.DictReader(f, dialect='excel-tab'))
print metacyc_compounds[0].keys()

['KEGG', 'Compounds', 'Monoisotopic-Molecular-Weight', 'Wikipedia', 'PubChem', 'Names', 'Common-Name', 'InChI', 'Object ID', 'LIPID MAPS']


In [3]:
""" contribution from Hans de Winter """
def _InitialiseNeutralisationReactions():
    patts= (
        # Imidazoles
        ('[n+;H]','n'),
        # Amines
        ('[N+;!H0]','N'),
        # Carboxylic acids and alcohols
        ('[$([O-]);!$([O-][#7])]','O'),
        # Thiols
        ('[S-;X1]','S'),
        # Sulfonamides
        ('[$([N-;X2]S(=O)=O)]','N'),
        # Enamines
        ('[$([N-;X2][C,N]=C)]','N'),
        # Tetrazoles
        ('[n-]','[nH]'),
        # Sulfoxides
        ('[$([S-]=O)]','S'),
        # Amides
        ('[$([N-]C=O)]','N'),
        )
    return [(Chem.MolFromSmarts(x),Chem.MolFromSmiles(y,False)) for x,y in patts]

_reactions=None
def NeutraliseCharges(mol, reactions=None):
    global _reactions
    if reactions is None:
        if _reactions is None:
            _reactions=_InitialiseNeutralisationReactions()
        reactions=_reactions
#     mol = Chem.MolFromSmiles(smiles)
    replaced = False
    for i,(reactant, product) in enumerate(reactions):
        while mol.HasSubstructMatch(reactant):
            replaced = True
            rms = AllChem.ReplaceSubstructs(mol, reactant, product)
            rms_smiles = Chem.MolToSmiles(rms[0])
            mol = Chem.MolFromSmiles(rms_smiles)
    if replaced:
        return (mol, True) #Chem.MolToSmiles(mol,True)
    else:
        return (mol, False)

In [4]:
all_compounds = []
for myCompound in metacyc_compounds:
    myMol = Chem.MolFromInchi(myCompound['InChI'])
    try:
        myCharge = Chem.GetFormalCharge(myMol)
        if  myCharge != 0:
            (myMol, neutralised) = NeutraliseCharges(myMol)
#             if neutralised == False:
#                 print neutralised, myCompound['Common-Name']
#                 print rdMolDescriptors.CalcMolFormula(neutral_mol)
        c = metob.Compound()
        c.InChI = myCompound['InChI']
        c.formula = rdMolDescriptors.CalcMolFormula(myMol)
        c.MonoIsotopic_molecular_weight = Descriptors.ExactMolWt(myMol)
        c.description = ''
        c.name = myCompound['Common-Name']
        all_compounds.append(c)

                
    except:
        print "Can not parse ", myCompound['Common-Name']


Can not parse  chlorite
Can not parse  methanetelluronate
Can not parse  choline sulfate
Can not parse  chlorate
Can not parse  tellurite
Can not parse  dichlorocarbene


In [5]:
#add them to the database
metob.store(all_compounds)

In [2]:
allCompounds = metob.retrieve('Compound')
print len(allCompounds)

ProgrammingError: (pymysql.err.ProgrammingError) (1146, u"Table 'meta_atlas.compounds_functional_sets' doesn't exist") [SQL: u'select * from compounds_functional_sets where source_id in ("0005ec9d4a8e4f1c9bd7cdee585a1ebc" , "000905053e76489e821e6ec2b6a0f23d" , "00152294f11f42b59f56e5a0d7f41828" , "0016099714c84e3582f453588cec78ab" , "0018f8a8564f4711b38cb4fe8f0d1c35" , "00204cdb42c44e9884849d617ba44aa4" , "0029ce6ff55b44468275b69597be364c" , "002c3a1b155b46afbf164b4be54d5074" , "00310b0b54d148b29de04dc2e7046c4b" , "00321867b300448f8e713d8cf057298b" , "0039312ddf4e475a8142fb5f9e528edd" , "003e89dcdeea4816a1906ace731b643f" , "003f27bd738642edac65637b5f83495d" , "004c63b06c1347feadc68ef03d4e190b" , "004fccc2ecef488083c5da9e6a5b3688" , "005094159b314c1cbdf67318754edf24" , "00537bc9927f45c18c69f00f0d499e75" , "005735c282894f2c9bf3942917c290a2" , "00585a91e72a44da836df410a8567a96" , "005a47a32d974c96b9f9ee05bae57d8f" , "005de7ba5dac4fb9a2a3373fc78591e4" , "00633e0ab40f40a5b31ff03d1988199b" ,

In [35]:
for c in allCompounds:
    print c.name
    print c.InChI

actinonin
InChI=1S/C19H35N3O5/c1-4-5-6-8-14(11-16(24)21-27)18(25)20-17(13(2)3)19(26)22-10-7-9-15(22)12-23/h13-15,17,23,27H,4-12H2,1-3H3,(H,20,25)(H,21,24)/t14-,15+,17+/m1/s1
actinomycin D
InChI=1S/C62H86N12O16/c1-27(2)42-59(84)73-23-17-19-36(73)57(82)69(13)25-38(75)71(15)48(29(5)6)61(86)88-33(11)44(55(80)65-42)67-53(78)35-22-21-31(9)51-46(35)64-47-40(41(63)50(77)32(10)52(47)90-51)54(79)68-45-34(12)89-62(87)49(30(7)8)72(16)39(76)26-70(14)58(83)37-20-18-24-74(37)60(85)43(28(3)4)66-56(45)81/h21-22,27-30,33-34,36-37,42-45,48-49H,17-20,23-26,63H2,1-16H3,(H,65,80)(H,66,81)(H,67,78)(H,68,79)
actinomycinate
InChI=1S/C62H106N12O18/c1-27(2)42(59(86)73-23-17-19-36(73)57(84)69(13)25-38(77)71(15)48(29(5)6)61(88)89)65-55(82)44(33(11)75)67-53(80)35-22-21-31(9)51-46(35)64-47-40(41(63)50(79)32(10)52(47)92-51)54(81)68-45(34(12)76)56(83)66-43(28(3)4)60(87)74-24-18-20-37(74)58(85)70(14)26-39(78)72(16)49(30(7)8)62(90)91/h21-22,27-30,33-34,36-39,42-45,48-49,57-62,75-78,84-91H,17-20,23-26,63H2,1-16H3,(H,65,8